In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from fem import install

install()

# **Lab 10 - Metodo agli Elementi Finiti (stazionario) - Parte 1**

Il metodo agli elementi finiti (FEM) è una tecnica di risoluzione numerica per equazioni alle derivate parziali, basata sulla discretizzazione di domini spaziali attraverso mesh poligonali (spesso e volentieri triangolari). Nel caso mono-dimensionale, in particolare, ciò si riduce all'introduzione di griglie spaziali.

La peculiarità del FEM è quella di risolvere il problema differenziali in *forma debole*, cioè passando da un'equazione puntuale (definita per ogni $x$ nel dominio), ad una variazionale (definita per ogni funzione test $v$). Per fare ciò, si fa leva su alcuni concetti di Analisi Funzionale, quali: spazi funzionali (Sobolev e Lebesgue), norme integrali, prodotti interni, forme bilineari, funzionali lineari, etc.

## Discretizzazione agli elementi finiti - **Mesh**

L'idea alla base del FEM è quella di discretizzare il dominio spaziale $\Omega$ introducendo una mesh $\mathcal{M}$ partizionata in *elementi*. Scelto un grado polinomiale $r$, quest'ultima viene utilizzata per costruire uno spazio elementi finiti

$$V_{h}\subset L^{2}(\Omega),$$

caratterizzato da tutte quelle funzioni $v_h:\Omega\to\mathbb{R}$ che sono polinomiali a tratti (di grado $r$), cioè, limitatamente ad ogni elemento della mesh, si possono scrivere come polinomi di grado $r$.

Nel caso Lagrangiano, questa costruzione è automaticamente associata, ad una collezione di nodi, $x_{1},\dots,x_{N_h}$, detti *gradi di libertà* (dofs). Questi ultimi, infatti, servono per l'interpolazione locale, che avviene elemento per elemento (similmente alle spline).

In [ ]:
from fem import Line, generate_mesh, FEspace, plot







In [ ]:
from fem import dofs



## Discretizzazione agli elementi finiti - **Funzioni**

Il vantaggio principale è che ogni funzione $f_h\in V_h$ si può rappresentare **univocamente** attraverso il vettore dei suoi valori nodali $\mathbf{f}_h$. Cioè, esiste una corrispondenza 1-a-1

$$V_h\ni f_h\;\;\longleftrightarrow\;\;\mathbf{f}_{h}\in \mathbb{R}^{N_h}$$

dove $\mathbf{f}_{h}=[f_h(x_1),\dots,f_h(x_{N_h})]$ è il vettore di valori nodali.
</br></br>
La corrispondenza è biunivoca perché: data $f_h$, il vettore $\mathbf{f}_h$ si calcola facilmente valutando $f_h$ nei nodi; viceversa, dato il vettore di valori nodali, basta interpolare localmente per ottenere $f_h$.

In [ ]:
from fem import interpolate




In [ ]:
plt.figure(figsize = (4, 3))
plot(fh, marker = '.', label = '$f_h$')

xplot = np.linspace(0, 1, 1000)
plt.plot(xplot, f(xplot), '--r', label = '$f$', alpha = 0.25)
plt.legend()
plt.show()

In [ ]:
from fem import dof2fun, fun2dof




<mark>**Esercizio 1**</mark></br>

Le funzioni di base $\varphi_{j}\in V_{h}$ sono quelle funzioni la cui rappresentazione in vettore dof corrisponde ai vettori della base canonica $\mathbf{e}_{j}=[0,0,\dots,1,\dots,0,0]$, dove "l'1" è in posizione $j$.

Si consideri la terza funzione di base, $j=3$, secondo l'ordinamento proposto da FEniCS.

1. Rappresentare graficamente $\varphi_j$,
2. Determinare $x$ tale che $\varphi_{j}(x)=1$.

NB: non fate confusione con l'indicizzazione, ricordate che in Python partiamo a contare da zero!

# Discretizzazione agli elementi finiti - **Funzionali lineari**

Poiché ogni funzione in $V_h$ si rappresenta univocamente con un vettore in $\mathbb{R}^{N_h}$, questo ci permette di rappresentare facilmente anche altri oggetti, tra cui i *funzionali lineari*. Infatti, si dimostra che ad ogni $\ell:V_h\to\mathbb{R}$ lineare corrisponde un $\mathbf{F}\in\mathbb{R}^{N_h}$ tale che</br></br>

$$\ell(v_h)=\mathbf{v}_h^\top\mathbf{F}\quad\quad\forall v_h\in V_h$$

dove $v_h\leftrightarrow\mathbf{v}_h$ come prima.
</br>
</br>
Di seguito un esempio per il funzionale lineare $\ell:\;v_h\mapsto \int x^2 v_h(x)dx$.

In [ ]:
from fem import assemble, dx






In [ ]:
vh = interpolate(lambda x: 1-x, V)
vh = fun2dof(vh) # passaggio a rappresentazione vettoriale

vh.T @ F # Equivalente a calcolare l(vh)!

# Discretizzazione agli elementi finiti - **Forme bilineari**

Analogamente a prima, si dimostra che per ogni forma bilineare $a:V_h\times V_h\to\mathbb{R}$ esiste una matrice $\mathbf{A}\in\mathbb{R}^{N_h\times N_h}$ tale che</br></br>

$$a(u_h,v_h)=\mathbf{v}_h^\top\mathbf{A}\mathbf{u}_h\quad\quad\forall u_h,v_h\in V_h.$$
</br>
</br>
Di seguito un esempio per la forma bilineare $a:\;(u_h,v_h)\mapsto\int u'_h(x)v_h(x)dx$.

In [ ]:
from fem import deriv






In [ ]:
uh = interpolate(lambda x: x**2, V)
vh = interpolate(lambda x: (1-x), V)

uh = fun2dof(uh)
vh = fun2dof(vh)

vh.T @ A @ uh # Equivalente a calcolare a(uh, vh)!

<mark>**Esercizio 2**</mark></br>

Usando lo spazio elementi finiti già costruito, assemblate la forma bilineare

$$m:\;(u,v)\mapsto \int uv dx$$

la cui matrice corrispondente, $\mathbf{M}$,  è detta *matrice di massa*.

Visualizzate la matrice $\mathbf{M}$: è simmetrica? è a dominanza diagonale per righe/colonne? è definita positiva?
</br>
</br>
NB: sfruttate il comando $\texttt{.todense()}$ per passare dal formato sparso a quello "pieno".

## Applicazione ai problemi ellittici

Grazie a queste rappresentazioni così efficaci, il FEM ci permette di risolvere equazioni differenziali (lineari) trasformandole in problemi algebrici (sistemi lineari). Vediamolo con un esempio.
</br>
</br>
Sia $\Omega=(a,b)$. Vogliamo risolvere il problema

$$-u'' = f \quad \text{in}\;\Omega,$$

complementato da condizioni di Dirichlet (dbc), $u(a)=\alpha$, $u(b)=\beta$, ai bordi del dominio. Abbiamo

- **Formulazione forte**: trovare $u\in \mathcal{C}^{2}(\Omega)$ soddisfacente le dbc e tale che</br></br>
$$-u''(x)=f(x)\quad\forall x\in\Omega.$$</br>

- **Formulazione debole**: trovare $u\in H^{1}(\Omega)$ soddisfacente le dbc e tale che</br></br>
$$\int_a^bu'v'dx=\int_a^bfvdx\quad\forall v\in H_0^1(\Omega).$$</br>

- **Problema di Galerkin**: trovare $u_h\in V_h$ soddisfacente le dbc e tale che</br></br>
$$\int_a^bu_h'v_h'dx=\int_a^bfv_hdx\quad\forall v_h\in V_h\cap H_0^1(\Omega).$$</br>


- **Formulazione algebrica**: trovare $\mathbf{u}_h\in\mathbb{R}^{N_h}$ soddisfacente le dbc e tale che</br></br>
$$\mathbf{A}\mathbf{u}_h = \mathbf{F}.$$</br>


- **Formulazione algebrica (con dbc)**: trovare $\mathbf{u}_h\in\mathbb{R}^{N_h}$ tale che</br></br>
$$\tilde{\mathbf{A}}\mathbf{u}_h = \tilde{\mathbf{F}}.$$</br>

L'ultimo step si ottiene modificando $\mathbf{A}$ e $\mathbf{F}$ in maniera opportuna, così da includere le condizioni al bordo. Ad es., se $j$ è la componente che fa riferimento al nodo $x_j=a$, si impone $F_j=\alpha$ e si sovrascrive la riga $j$-esima di $\mathbf{A}$ ponendo tutti 0 fuorché in posizione $j$ (dove si mette un 1).
</br>
</br>
Tutto ciò ci permette di trovare $\mathbf{u}_h$, e quindi $u_h$, risolvendo un sistema lineare.


<mark>**Esercizio 3**</mark></br>

Si consideri il seguente problema ellittico,</br></br>

$$\begin{cases}-u'' = e^{2x}\left(3\sin x + 4\cos x\right) & x\in(0,2\pi)\\
\\u(0)=u(2\pi)=0,
\end{cases}$$
</br>
Si risolva numericamente il problema differenziale implementando il metodo agli elementi finiti con $h=0.01$ ed $r=1$.

In [ ]:
# Mesh e spazio elementi finiti





# Assemblaggio del termine noto
f = lambda x: np.exp(2*x)*(3*np.sin(x) + 4*np.cos(x))





# Assemblaggio della matrice del sistema





In [ ]:
# Aggiustamento delle condizioni al bordo
from fem import DirichletBC

def isLeftNode(x):
  return x < 1e-12

def isRightNode(x):
  return x > 2*np.pi - 1e-12

dbc1 = DirichletBC(isLeftNode,  0.0)
dbc2 = DirichletBC(isRightNode, 0.0)

from fem import applyBCs
A = applyBCs(A, V, dbc1, dbc2)
F = applyBCs(F, V, dbc1, dbc2)

In [ ]:
# Risoluzione del sistema lineare
from scipy.sparse.linalg import spsolve






<mark>**Esercizio 4**</mark></br>

Si consideri il problema alle derivate parziali descritto precedentemente. La soluzione esatta di tale problema è

$$u(x)=-e^{2x}\sin(x).$$

Se $u_h$ è la soluzione elementi finiti (come funzione, non come vettore!), il seguente pezzo di codice

    from fem import L2error
    uex = lambda x: -np.exp(2*x)*np.sin(x)
    L2error(uex, uh, domain)

vi permette di calcolare l'errore in norma $L^{2}$, definito dalla formula $\sqrt{\int_a^b|u(x)-u_h(x)|^2dx}$.
</br></br>
Avendo fissato il grado polinomiale della discretizzazione agli elementi finiti, $r=1$, si calcoli l'errore in norma $L^{2}$ tra la soluzione FEM e la soluzione esatta al variare del passo di discretizzazione $h=0.2,0.1,0.05,0.025$. Plottare graficamente l'andamento dell'errore: i risultati sono coerenti con la teoria?

In [ ]:
from fem import L2error

uex = lambda x: -np.exp(2*x)*np.sin(x)
L2error(uex, uh, domain)

In [ ]:
r = 1
h = np.array([0.2, 0.1, 0.05, 0.025])









# **Extra** - FEM in 2D

In realtà, tutto quello che abbiamo visto si adatta istantaneamente al caso multi-dimensionale! Le grigle diventano mesh, i sotto-intervalli diventano elementi (spesso triangolari), ed i vari operatori differenziali trovano la loro controparte (gradiente, divergenza, rotore... etc.). Di seguito, un esempio di problema ellittico in 2D, per i più curiosi.

In [ ]:
from fem import Rectangle, Circle
domain = Rectangle((-1, -1), (1, 1)) - Circle((0, 0), 0.5)
mesh = generate_mesh(domain, stepsize = 0.1, structured = True)
V = FEspace(mesh, 1)

plt.figure(figsize = (8, 4))
plt.subplot(1, 2, 1)
plot(mesh, title = "Mesh")
plt.subplot(1, 2, 2)
plot(V, title = "Posizione dei dofs")

In [ ]:
from fem import inner, grad

f = lambda x, y: 1.0
fh = interpolate(f, V)
l = lambda v: fh*v*dx

isOnCircle = lambda x, y: (x**2 + y**2)**0.5 < 0.5 + 1e-12
isOnSquare = lambda x, y: not isOnCircle(x, y)

dbc1 = DirichletBC(isOnCircle, 0.0)
dbc2 = DirichletBC(isOnSquare, lambda x, y: np.sin(np.pi*x)-y+1)

a = lambda u, v: inner(grad(u), grad(v))*dx

F = applyBCs(assemble(l, V), V, dbc1, dbc2)
A = applyBCs(assemble(a, V), V, dbc1, dbc2)

uh = spsolve(A, F)
uh = dof2fun(uh, V)

plt.figure(figsize = (4, 4))
plot(uh, title = "Soluzione elementi finiti")